In [ ]:
!pip install krippendorff

In [ ]:
import json
import copy
import pandas as pd
import re
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
import krippendorff as kd
import numpy as np

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/assignments_from_pool_42774131__05-02-2024.tsv", sep="\t")
df = df[df['ASSIGNMENT:status']=='APPROVED']

In [ ]:
!pip install crowd-kit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
from crowdkit.metrics.data._classification import alpha_krippendorff

In [ ]:
from collections import Counter

toloker2count = dict(Counter(pd.DataFrame.from_dict(tasks_labels).worker.values.tolist()).most_common())

In [ ]:
toloker2count

{'e31a149b5bd171e620d781a8c04e8450': 100,
 'd8979dd5effd69d538d497789485a841': 100,
 '2c17040f134eeb48660a38e63e336643': 100,
 '5d0f5b447fe8638352951db6bfc13956': 95,
 'b75db3f4d24677cc9ee1f10603052701': 85,
 '437cb936254a18312f9c5cb2ef17310b': 80,
 'f085fe283bae100d65873376d531c985': 80,
 'ddb4eeb316dcdb946c1e09cd8919c025': 80,
 '9a8928d0d44faa52cf9e7118122d68a3': 75,
 '5a8b25b7b826205782008b16b2b09ca6': 75,
 'cf77cdcb2352f568a5f00f39d4be2621': 65,
 '72a958125bb2b80345a30499e953d0a4': 65,
 '8b79bc8a2f3819ac87fdd93372eede7e': 65,
 '7560eaef8a8637da394977d877b21ee8': 60,
 'ad486b05c2371539edf2f43c139db430': 55,
 '1b84f371be5de55b14ff6d8207a8905f': 55,
 '66e577cd94e4013ef400ecc57cd28034': 50,
 'e6eb5e05e7207c899ccf1354f565ab37': 50,
 'e34701fca8125e5a45ca1e223f8d04d0': 50,
 'd076f83c6a19012bfb580566ba036e7c': 50,
 '55c0a8288b3924eb0052b5ec6ec3c90f': 50,
 'e2aeceb4754e428ebe187e85cfdb86ef': 45,
 'b716768874af4d849781e9aaa0654ac1': 45,
 'e67ea8f92ce6d033ce6cc2536209e4fb': 45,
 'b54d0d44193

In [ ]:
tasks = {}
tasks_labels = []
for index, row in df.iterrows():
    tasks_labels.append({'task': row['ASSIGNMENT:task_id'], 'worker': row['ASSIGNMENT:worker_id'], 'label': int(row['OUTPUT:R-4'])})
    if row["ASSIGNMENT:task_id"] not in tasks:
        tasks[row["ASSIGNMENT:task_id"]] = {}
        tasks[row["ASSIGNMENT:task_id"]]["text"] = row["INPUT:input"]
        tasks[row["ASSIGNMENT:task_id"]]["workers"] = {row["ASSIGNMENT:worker_id"]:row["OUTPUT:R-4"]}
    else:
        tasks[row["ASSIGNMENT:task_id"]]["workers"][row["ASSIGNMENT:worker_id"]] = row["OUTPUT:R-4"]

In [ ]:
import json
with open("output_annotation.json", 'w') as w:
    json.dump(tasks, w)

In [ ]:
pd.DataFrame.from_records(tasks_labels)

,task,worker,label
0,00028cae73--65bfa3f40cadac2361249abc,d7c8ba18b024aadad3c03dc12bceface,2
1,00028cae73--65bfa3a1a15018044b04914f,d7c8ba18b024aadad3c03dc12bceface,2
2,00028cae73--65bfa3f80cadac236124a05f,d7c8ba18b024aadad3c03dc12bceface,2
3,00028cae73--65bfa3f30cadac23612498c8,d7c8ba18b024aadad3c03dc12bceface,2
4,00028cae73--65bfa3f50cadac2361249b93,d7c8ba18b024aadad3c03dc12bceface,2
...,...,...,...
6404,00028cae73--65bfa3f80cadac236124a041,8dc6cecab5a1e798269b109e83f64621,2
6405,00028cae73--65bfa3f30cadac236124994e,8dc6cecab5a1e798269b109e83f64621,2
6406,00028cae73--65bfa3f70cadac2361249f6e,8dc6cecab5a1e798269b109e83f64621,2
6407,00028cae73--65bfa3f70cadac2361249e1f,8dc6cecab5a1e798269b109e83f64621,2


In [ ]:
alpha_krippendorff(pd.DataFrame.from_records(tasks_labels))

0.2686509960442679

In [ ]:
from collections import Counter

js = []
for i,j in Counter(pd.DataFrame.from_dict(tasks_labels).task.values.tolist()).most_common():
    js.append(j)

Counter(js)

Counter({76: 1,
         72: 2,
         71: 1,
         70: 1,
         68: 1,
         66: 4,
         63: 2,
         62: 2,
         61: 1,
         60: 1,
         59: 2,
         51: 1,
         49: 1,
         3: 1709})

In [ ]:
from nltk.tokenize import NLTKWordTokenizer

In [ ]:
tags = {1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 0.0: 0}

In [ ]:
df_raters = pd.DataFrame()

for n_sent, task in enumerate(tasks):
    text = tasks[task]['text']
    workers =  tasks[task]['workers']
    worker2annotation = {}
    worker2annotation['text'] = text
    worker2annotation['n_sent'] = n_sent
    for worker, label in workers.items():
        worker2annotation[worker] = label

    df_raters = df_raters.append(worker2annotation, ignore_index=True)

<ipython-input-30-f5a091d3314f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raters = df_raters.append(worker2annotation, ignore_index=True)
<ipython-input-30-f5a091d3314f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raters = df_raters.append(worker2annotation, ignore_index=True)
<ipython-input-30-f5a091d3314f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raters = df_raters.append(worker2annotation, ignore_index=True)
<ipython-input-30-f5a091d3314f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raters = df_raters.append(worker2annotation, ignore_index=True)
<ipython-input-30-f5a091d3314f>:12: FutureWarnin

In [ ]:
df_raters = df_raters.astype({'n_sent': 'int32'})

In [ ]:
df_raters

,text,n_sent,d7c8ba18b024aadad3c03dc12bceface,31e4bb0217a21ef6451faaee8847d7c1,b0e385e880cb3f5033308f1a9d63e5c4,9a8928d0d44faa52cf9e7118122d68a3,646ef569491540d55e1f3140841a1a15,2273fc122c3fd60f9e096cf0afaeabb9,5f1770836609d188639b9b73f25500d8,a7cda650641a4271fba8f12bceef3cbf,...,9f92499d14c6fdba6881e7dc9e6edb97,6c0aed0f589955fe8a49de77e935f62c,d04cf36ea1950c41c1477f00bd29e0a2,0cf8c7a691d536b732042ff3db2f1023,2e705095056dec78df05515f62656880,4e68b29b49a21c8f55390154db8e2756,7a91a16c8662352508c0f2cfde2f6f42,b3d4859e6d92a76c007fc8e7d9b2aa92,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,**Family Guy** vs **The Simpsons**\r\n\r\n*Rat...,0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,**Cement** vs **Concrete**\r\n\r\n*Rate the se...,1,2.0,NaN,NaN,3.0,2.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,**chocolate** vs **tea**\r\n\r\n*Rate the sent...,2,2.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,**BMW** vs **Audi**\r\n\r\n*Rate the sentence ...,3,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,**kids** vs **adults**\r\n\r\n*Rate the senten...,4,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,**concrete** vs **wood**\r\n\r\n*Rate the sent...,1724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1725,**perl** vs **python**\r\n\r\n*Rate the senten...,1725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1726,**skiing** vs **snowboarding**\r\n\r\n*Rate th...,1726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,**baseball** vs **softball**\r\n\r\n*Rate the ...,1727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Count Fleiss' kappa

In [ ]:
sentence = df_raters.loc[:, df_raters.columns != 'text']

### Here we select the annotators

In [ ]:
annotators = list(df_raters.columns)[2:]
sentence = sentence[annotators]

In [ ]:
sentence = sentence[annotators]
all_data = sentence[~sentence.isnull().any(axis=1)]
print(all_data)
if len(all_data) > 0:
    print(fleiss_kappa(aggregate_raters(all_data)[0], method='fleiss'))

Empty DataFrame
Columns: [d7c8ba18b024aadad3c03dc12bceface, 31e4bb0217a21ef6451faaee8847d7c1, b0e385e880cb3f5033308f1a9d63e5c4, 9a8928d0d44faa52cf9e7118122d68a3, 646ef569491540d55e1f3140841a1a15, 2273fc122c3fd60f9e096cf0afaeabb9, 5f1770836609d188639b9b73f25500d8, a7cda650641a4271fba8f12bceef3cbf, ad486b05c2371539edf2f43c139db430, 437cb936254a18312f9c5cb2ef17310b, 9e305e89ce6dd3f65e2fb46d70a9f873, fddc310e0c2a2d036d76c6c36f17c352, fff056f4e0d75d477c80d7b94b03785a, 66e577cd94e4013ef400ecc57cd28034, e6eb5e05e7207c899ccf1354f565ab37, 1b84f371be5de55b14ff6d8207a8905f, e31a149b5bd171e620d781a8c04e8450, cf77cdcb2352f568a5f00f39d4be2621, 4423213b21add4b265e284a655029d8a, e2aeceb4754e428ebe187e85cfdb86ef, 7560eaef8a8637da394977d877b21ee8, 728d61a6e46622c761820aaa258ddbd8, e34701fca8125e5a45ca1e223f8d04d0, b2a98453c0a9fd971cef888eb34cb794, 5d0f5b447fe8638352951db6bfc13956, d8979dd5effd69d538d497789485a841, c35233cdc004c075d0762ad4e4dea42e, 72a958125bb2b80345a30499e953d0a4, a74bfbea81e153a521cb9c

## Count Krippendorf's Alpha

In [ ]:
sentence.T

,0,1,2,3,4,5,6,7,8,9,...,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728
d7c8ba18b024aadad3c03dc12bceface,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31e4bb0217a21ef6451faaee8847d7c1,2.0,NaN,3.0,3.0,2.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b0e385e880cb3f5033308f1a9d63e5c4,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9a8928d0d44faa52cf9e7118122d68a3,NaN,3.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646ef569491540d55e1f3140841a1a15,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4e68b29b49a21c8f55390154db8e2756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7a91a16c8662352508c0f2cfde2f6f42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b3d4859e6d92a76c007fc8e7d9b2aa92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355e2bf3c1d973f919f3a979d966b25c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
kd.alpha(sentence.T, level_of_measurement='interval')

0.3350446915451717

In [ ]:
sentence

,355e2bf3c1d973f919f3a979d966b25c,7d77e5454ea232d0be01868a0b4be267
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1724,NaN,NaN
1725,NaN,NaN
1726,NaN,NaN
1727,NaN,NaN


## Count Cohen's kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score
import itertools

In [ ]:
cohen = []

for a1, a2 in itertools.combinations(annotators, 2):
    sentence = df_raters[[a1, a2]]
    all_data = sentence[~sentence.isnull().any(axis=1)]
    list1 = [tags[i] for i in all_data[a1].values.tolist()]
    list2 = [tags[i] for i in all_data[a2].values.tolist()]
    if str(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4])) != 'nan':
        cohen.append(cohen_kappa_score(list1, list2, labels=[0,1,2,3,4]))

In [ ]:
np.mean(cohen), np.median(cohen)

(0.16959827587164003, 0.0)

In [ ]:
from collections import Counter

In [ ]:
c = 0
with open("labelling_dataset_ru.tsv", 'w') as w:
    w.write(f"text\ttag\n")
    for _, subset in df_raters.groupby('n_sent'):
        c += 1
        lines = pd.DataFrame(subset).values.tolist()
        for line in lines:
            text = line[0] #.replace("***", "")
            labels = [i for i in line[2:] if not i is np.nan]
            label = Counter(labels).most_common()[0][0]
            w.write(f"{text}\t{label}\n")
c

1370

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("labelling_dataset_ru.tsv", sep='\t')

In [ ]:
df.tag.value_counts()

NONE      964
BETTER    296
WORSE     110
Name: tag, dtype: int64

In [ ]:
with open("labelling_dataset_ru_not_mixed.tsv", 'w') as w:
    w.write(f"nb_sent\tword\ttag\n")
    for rater in ["3b3a0c5c761f50ae205a52aee0a3572e", "09f97040315a699d82a34e22dd1193b6", "8e9beb3e5b8aa496af39f03c3f6f9513"]:
        subset = df_raters[["token", "n_sent", rater]]
        lines = subset.values.tolist()
        for line in lines:
            w.write(f"{line[1]}\t{line[0]}\t{line[2]}\n")